# Given notebook performs performance estimation of first DBSCAN
* **fp**	False positive: Amount of clusters not having any RFID records
* **fn**	False negative: Amount of RFID not belong to any cluster
* **tp**	True positive: Amount of RFID having cluster id
* **RFID_coverage_rate**	tp/(fp+fn+tp)

In [1]:
import os
import numpy as np
import pandas as pd
import pymongo
from sklearn.cluster import DBSCAN
import plotly.express as px
import geopandas as gpd
from multiprocessing import Pool
from ClusteringPipeline import do_clustering
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'


# ML models paths
IS_MOVING = '/models/vehicle_movement_detector.sav'
SPEED_REHAB_MODEL = '/models/velocity_to_speed.sav'

# DBSCAN parameters
MIN_SAMPLES = 15
EPS = 130
MIN_POINT_AMT_RFID = 1

# PERFORMANCE FOLDER:
O_GPS_CLUSTERED_DIR = '/Data/Outputs/GPS_CLUSTERED_JS_SE/'

In [3]:
!mkdir -p /Data/Outputs/GPS_CLUSTERED_JS_SE/

# Data

### 1. route vehicle assignment

In [4]:
I_ROUTE_VEHICLE_ASSIGNMENT = '/Data/Intermediate/assignments/route_vehicle_assignments_october.csv'

route_vehic_assign = pd.read_csv(I_ROUTE_VEHICLE_ASSIGNMENT)
route_vehic_assign = route_vehic_assign[route_vehic_assign['route_type'] == 'JS-SE'] \
    .groupby(['route_type']).agg({'vehicle_id': 'unique'}).reset_index()

route_vehic_assign = route_vehic_assign.explode('vehicle_id')
route_vehic_assign.head()

,route_type,vehicle_id
0,JS-SE,XE-5638S
0,JS-SE,XE-5577J
0,JS-SE,XE-5629T


### 2. Map matching report

In [5]:
I_MAP_MATCHING_REPORT = '/Data/Intermediate/MapMatchingReports/MapMatching_october2020.csv'
map_matching_report = pd.read_csv(I_MAP_MATCHING_REPORT)

# Preprocessing
map_matching_report['truck_id'] = map_matching_report['truck_id'].str.replace(' ', '-')
map_matching_report = map_matching_report.rename(columns={'truck_id': 'vehicle_id'})

# Selection JS-SE
map_matching_report = map_matching_report.merge(route_vehic_assign[['vehicle_id']], on=['vehicle_id'])

### 3. GPS

In [6]:
S_GPS_RECORDS_DIR = '/Data/Source/OctoberGPS/'

FILES = map_matching_report['csv_file'].to_list()
SOURCE_GPS_PATHS = (S_GPS_RECORDS_DIR + map_matching_report['csv_file']).to_list()

## 1. Setup clustering

In [7]:
def read_data(args):
    args['gps'] = pd.read_csv(args['source_path'], parse_dates=['time'])
    args['gps'] = args['gps'].loc[~args['gps'].unixtime.duplicated(keep='last'), :] \
        .reset_index(drop=True).copy()
    args['gps'].truck_id = args['gps'].truck_id.str.replace(' ', '-')
    return args


def clusterize(args):
    results = do_clustering(args['gps'], min_samples=args['min_samples'],
                       eps=args['eps'],
                       is_moving_log_reg_path = args['is_moving_log_reg_path'],
                       speed_rehab_model_path = args['speed_rehab_model_path'],
                       fname = args['fname'])
    args['gps'] = results['df'].copy()
    del results
    return args


def export_clustered_gps(args):
    fpath = args['merge_dir'] + args['fname']
    args['gps'].to_csv(fpath, index=False)
    del args

def score(args):

    PIPELINE = [
        read_data,
        clusterize,
        export_clustered_gps,
    ]
    
    fname = args['source_path'].rsplit('/', 1)[1]
    
    for func in PIPELINE:
        args = func(args)
    print(f"{fname}: is done!")
    return args

## Arguments preparation

In [8]:
N_ARGS = len(SOURCE_GPS_PATHS)
args = list(map(lambda src_path, # read_data
                min_samples, eps, is_moving_log_reg_path, # DBSCAN
                speed_rehab_model_path, fname, # DBSCAN
                merge_dir:  
                {
                    'source_path': src_path,
                    'min_samples': min_samples,
                    'eps': eps,
                    'is_moving_log_reg_path': is_moving_log_reg_path,
                    'speed_rehab_model_path': speed_rehab_model_path,
                    'fname': fname,
                    'merge_dir': merge_dir
                },
                SOURCE_GPS_PATHS,
                [MIN_SAMPLES]*N_ARGS,
                [EPS]*N_ARGS,
                [IS_MOVING]*N_ARGS,
                [SPEED_REHAB_MODEL]*N_ARGS,
                FILES,
                [O_GPS_CLUSTERED_DIR]*N_ARGS
               ))

In [9]:
%%time
with Pool(5) as pool:
    outputs = pool.starmap(score, zip(args))

/usr/local/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LinearRegression from version 0.23.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LinearRegression from version 0.23.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use

XE-5638S_2020-10-30.csv: is done!


/usr/local/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LinearRegression from version 0.23.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


XE-5638S_2020-10-26.csv: is done!

/usr/local/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


...

XE-5629T_2020-10-27.csv: is done!


/usr/local/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LinearRegression from version 0.23.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


XE-5629T_2020-10-25.csv: is done!
XE-5629T_2020-10-20.csv: is done!
XE-5629T_2020-10-4.csv: is done!


/usr/local/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LinearRegression from version 0.23.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LinearRegression from version 0.23.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use

XE-5629T_2020-10-5.csv: is done!
XE-5629T_2020-10-14.csv: is done!
CPU times: user 157 ms, sys: 102 ms, total: 259 ms
Wall time: 30.4 s
